# Migrating to modern conversation management

Follow this guide if you're trying to migrate off one of the old memory classes listed below:


| Memory Type                          | Description                                                                                                                                          |
|---------------------------------------|------------------------------------------------------------------------------------------------------------------------------------------------------|
| `ConversationBufferMemory`            | A basic memory implementation that simply stores the conversation history.                                                                            |
| `ConversationStringBufferMemory`      | A special case of `ConversationBufferMemory` designed for LLMs and no longer relevant.                                                                |
| `ConversationBufferWindowMemory`      | Keeps the last `n` turns of the conversation. Drops the oldest turn when the buffer is full.                                                          |
| `ConversationTokenBufferMemory`       | Keeps only the most recent messages in the conversation under the constraint that the total number of tokens in the conversation does not exceed a certain limit. |
| `ConversationSummaryMemory`           | Continually summarizes the conversation history. The summary is updated after each conversation turn. The abstraction returns the summary of the conversation history. |
| `ConversationSummaryBufferMemory`     | Provides a running summary of the conversation together with the most recent messages in the conversation under the constraint that the total number of tokens in the conversation does not exceed a certain limit. |
| `VectorStoreRetrieverMemory`          | Stores the conversation history in a vector store and retrieves the most relevant parts of past conversation based on the input.                        |


The goal of managing conversation history is to store and retrieve it in a way that maximizes its usefulness for a chat model. This often requires trimming or summarizing the conversation to retain the most relevant details while ensuring it fits within the model's context window.

LangChain v0.0.x offered several methods for managing conversation history. While these memory abstractions were helpful, they had limitations that made them less effective for real-world conversational AI. Many were developed before the advent of chat models, making them poorly suited for such applications. Additionally, they lacked robust solutions for handling multi-user, multi-conversation scenarios—an essential requirement for practical conversational AI systems.

The methods for handling conversation history using existing modern primitives are:

1. Using [LangGraph persistence](https://langchain-ai.github.io/langgraph/how-tos/persistence/) along with appropriate processing of the message history
2. Using LCEL with [RunnableWithMessageHistory](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html) combined with appropriate processing of the message history.

Most users will find [LangGraph persistence](https://langchain-ai.github.io/langgraph/how-tos/persistence/) both easier to use and easier to configure than the equivalent LCEL, especially for more complex use cases.

Some advantages to using LangGraph persistence:

- LangGraph persistence is much more powerful than simple chat memory -- it lets you save and resume complex state at any time for error recovery, human-in-the-loop workflows, time travel interactions, and more.
- LangGraph persistence is built to support multi user, multi thread scenarios and will work well in production settings.
- Both LLMs and chat models are fully supported.
- Users have granular control over how memory is updated.


:::hint

LangChain provides a number of useful helper utilties to process message history. Please see the following 
how to guides to learn more:

* [How to trim messages](https://python.langchain.com/docs/how_to/trim_messages)
* [How to filter messages](https://python.langchain.com/docs/how_to/filter_messages/)
* [How to merge message runs](https://python.langchain.com/docs/how_to/merge_message_runs/)

:::


# Migrating off ConversationBufferMemory / ConversationStringBufferMemory

[ConversationBufferMemory](https://api.python.langchain.com/en/latest/memory/langchain.memory.buffer.ConversationBufferMemory.html)
and [ConversationStringBufferMemory](https://python.langchain.com/api_reference/langchain/memory/langchain.memory.buffer.ConversationStringBufferMemory.html)
 were used to keep track of a conversation between a human and an ai asstistant without any additional processing. 
 
The `ConversationStringBufferMemory` is equivalent to `ConversationBufferMemory` but was targeting LLMs that were not chat models.

In [1]:
%%capture --no-stderr
%pip install --upgrade --quiet langchain-openai langchain langchain-community

In [2]:
import os
from getpass import getpass

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass()

## LLMChain / ConversationChain

This section shows how to migrate off `ConversationBufferMemory` or `ConversationStringBufferMemory` that's used together with either an `LLMChain` or a `ConversationChain`.

### Legacy

Below is example usage of `ConversationBufferMemory` with an `LLMChain` or an equivalent `ConversationChain`.

<details open>

In [3]:
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain_core.messages import SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate(
    [
        SystemMessage(content="You are a helpful assistant."),
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)

# highlight-start
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
# highlight-end

legacy_chain = LLMChain(
    llm=ChatOpenAI(),
    prompt=prompt,
    # highlight-next-line
    memory=memory,
)

legacy_result = legacy_chain.invoke({"text": "my name is bob"})
print(legacy_result)

legacy_result = legacy_chain.invoke({"text": "what was my name"})

/tmp/ipykernel_2459205/1840360377.py:24: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  legacy_chain = LLMChain(


{'text': 'Nice to meet you, Bob! How can I assist you today?', 'chat_history': [HumanMessage(content='my name is bob', additional_kwargs={}, response_metadata={}), AIMessage(content='Nice to meet you, Bob! How can I assist you today?', additional_kwargs={}, response_metadata={})]}


In [4]:
legacy_result["text"]

'Your name is Bob. How can I assist you further, Bob?'

</details>

### LangGraph

The example below shows how to use LangGraph to implement a `ConversationChain` or `LLMChain` with `ConversationBufferMemory`.

This example assumes that you're already somewhat familiar with `LangGraph`. If you're not, then please see the [LangGraph Quickstart Guide](https://langchain-ai.github.io/langgraph/tutorials/introduction/) for more details.

`LangGraph` offers a lot of additional functionality (e.g., time-travel and interrupts) and will work well for other more complex (and realistic) architectures.

<details open>

In [5]:
from langchain_core.messages import HumanMessage
import uuid
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from IPython.display import Image, display

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)

# Define a chat model
model = ChatOpenAI()


# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    # We return a list, because this will get added to the existing list
    return {"messages": response}


# Define the two nodes we will cycle between
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)


# Adding memory is straight forward in langgraph!
# highlight-next-line
memory = MemorySaver()

app = workflow.compile(
    # highlight-next-line
    checkpointer=memory
)


# highlight-start
# The thread id is a unique key that identifies
# this particular conversation.
# We'll just generate a random uuid here.
thread_id = uuid.uuid4()
config = {"configurable": {"thread_id": thread_id}}
# highlight-end

input_message = HumanMessage(content="hi! I'm bob")
for event in app.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

# Here, let's confirm that the AI remembers our name!
config = {"configurable": {"thread_id": "2"}}
input_message = HumanMessage(content="what was my name?")
for event in app.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

hi! I'm bob
================================== Ai Message ==================================

Hello, Bob! How can I assist you today?
================================ Human Message =================================

what was my name?
================================== Ai Message ==================================

I'm sorry, but I do not have the ability to know your name as I am an AI assistant.


</details>

### LCEL RunnableWithMessageHistory

Alternatively, if you have a simple chain, you can wrap the chat model of the chain within a [RunnableWithMessageHistory](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html).

Please refer to [this how to guide](/docs/how_to/message_history/) for more information.

## Agent Executor with a pre-built agent

This example shows usage of an Agent Executor with a pre-built agent constructed using the [create_tool_calling_agent](https://api.python.langchain.com/en/latest/agents/langchain.agents.tool_calling_agent.base.create_tool_calling_agent.html) function.

If you are using one of the [old LangChain pre-built agents](https://python.langchain.com/v0.1/docs/modules/agents/agent_types/), you should be able
to replace that code with the new [langgraph pre-built agent](https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/) which leverages
native tool calling capabilities of chat models and will likely work better out of the box.

### Legacy Usage

<details open>

In [6]:
from langchain import hub
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.memory import ConversationBufferMemory
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0)


@tool
def get_user_age(name: str) -> str:
    """Use this tool to find the user's age."""
    # This is a placeholder for the actual implementation
    if "bob" in name.lower():
        return "42 years old"
    return "41 years old"


tools = [get_user_age]

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")

# Instantiate memory
# highlight-start
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
# highlight-end

# Create an agent
agent = create_tool_calling_agent(model, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    # highlight-next-line
    memory=memory,  # Pass the memory to the executor
)

# Verify that the agent can use tools
print(agent_executor.invoke({"input": "hi! my name is bob what is my age?"}))
print()
# Verify that the agent has access to conversation history.
# The agent should be able to answer that the user's name is bob.
print(agent_executor.invoke({"input": "do you remember my name?"}))

/home/eugene/.pyenv/versions/3.11.4/envs/core/lib/python3.11/site-packages/langsmith/client.py:5301: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  prompt = loads(json.dumps(prompt_object.manifest))


{'input': 'hi! my name is bob what is my age?', 'chat_history': [HumanMessage(content='hi! my name is bob what is my age?', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello Bob! You are 42 years old. If you need any more assistance or have any other questions, feel free to ask!', additional_kwargs={}, response_metadata={})], 'output': 'Hello Bob! You are 42 years old. If you need any more assistance or have any other questions, feel free to ask!'}

{'input': 'do you remember my name?', 'chat_history': [HumanMessage(content='hi! my name is bob what is my age?', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello Bob! You are 42 years old. If you need any more assistance or have any other questions, feel free to ask!', additional_kwargs={}, response_metadata={}), HumanMessage(content='do you remember my name?', additional_kwargs={}, response_metadata={}), AIMessage(content='Yes, your name is Bob. How can I assist you further, Bob?', additional_kwarg

</details>

### LangGraph

This example shows how to add memory to the pre-built react agent in langgraph. This is an agent for which you only need to supply the tool and prompt.

After looking through the snippet below, you can reference [this guide](https://langchain-ai.github.io/langgraph/how-tos/create-react-agent-memory/) for more details.

<details open>

In [7]:
import uuid

from langchain_core.messages import HumanMessage
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent


@tool
def get_user_age(name: str) -> str:
    """Use this tool to find the user's age."""
    # This is a placeholder for the actual implementation
    if "bob" in name.lower():
        return "42 years old"
    return "41 years old"


# highlight-next-line
memory = MemorySaver()
model = ChatOpenAI()
app = create_react_agent(
    model,
    tools=[get_user_age],
    # highlight-next-line
    checkpointer=memory,
)

# highlight-start
# The thread id is a unique key that identifies
# this particular conversation.
# We'll just generate a random uuid here.
thread_id = uuid.uuid4()
config = {"configurable": {"thread_id": thread_id}}
# highlight-end

# Tell the AI that our name is Bob, and ask it to use a tool to confirm
# that it's capable of working like an agent.
input_message = HumanMessage(content="hi! I'm bob. What is my age?")

for event in app.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

# Confirm that the chat bot has access to previous conversation
# and can respond to the user saying that the user's name is Bob.
input_message = HumanMessage(content="do you remember my name?")

for event in app.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

hi! I'm bob. What is my age?
================================== Ai Message ==================================
Tool Calls:
  get_user_age (call_ewWp6keHVouwJyM4lRhk4X25)
 Call ID: call_ewWp6keHVouwJyM4lRhk4X25
  Args:
    name: bob
================================= Tool Message =================================
Name: get_user_age

42 years old
================================== Ai Message ==================================

Bob, you are 42 years old.
================================ Human Message =================================

do you remember my name?
================================== Ai Message ==================================

Yes, your name is Bob.


In [8]:
# Now, let's confirm that if we use a different thread
# that the bot will not know what our name is since
# it's a new conversation!
config = {"configurable": {"thread_id": "123456789"}}

input_message = HumanMessage(content="hi! do you remember my name?")

for event in app.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

hi! do you remember my name?
================================== Ai Message ==================================

Hello! Yes, I remember your name. It's great to see you again! How can I assist you today?


</details>

## Implementing Conversation History Processing

Each of the following memory types applies specific logic to handle the conversation history:

| Memory Type                          | Description                                                                                                                                          |
|---------------------------------------|------------------------------------------------------------------------------------------------------------------------------------------------------|
| `ConversationBufferWindowMemory`      | Keeps the last `n` turns of the conversation. Drops the oldest turn when the buffer is full.                                                          |
| `ConversationTokenBufferMemory`       | Keeps only the most recent messages in the conversation under the constraint that the total number of tokens in the conversation does not exceed a certain limit. |
| `ConversationSummaryMemory`           | Continually summarizes the conversation history. The summary is updated after each conversation turn. The abstraction returns the summary of the conversation history. |
| `ConversationSummaryBufferMemory`     | Provides a running summary of the conversation together with the most recent messages in the conversation under the constraint that the total number of tokens in the conversation does not exceed a certain limit. |

The general approach involves writing the necessary logic for processing conversation history and integrating it at the correct point.

We’ll start by building a simple processor using LangChain's built-in [trim_messages](https://python.langchain.com/api_reference/core/messages/langchain_core.messages.utils.trim_messages.html) function, and then demonstrate how to integrate it into your application.

You can later replace this basic setup with more advanced logic tailored to your specific needs.


:::important

We’ll begin by exploring a straightforward method that involves applying processing logic to the entire conversation history.

While this approach is easy to test and implement, it has a downside: as the conversation grows, s
o does the latency, since the logic is re-applied to all previous exchanges at each turn.

More advanced strategies focus on incrementally updating the conversation history to avoid redundant processing.

For instance, the langgraph [how-to guide on summarization](https://langchain-ai.github.io/langgraph/how-tos/memory/add-summary-conversation-history/) demonstrates
how to maintain a running summary of the conversation while discarding older messages, ensuring they aren't re-processed during later turns.
:::


### ConversationBufferWindowMemory, ConversationTokenBufferMemory

<details open>

In [12]:
from langchain_core.messages import (
    AIMessage,
    BaseMessage,
    HumanMessage,
    SystemMessage,
    trim_messages,
)


full_message_history = [
    SystemMessage("you're a good assistant, you always respond with a joke."),
    HumanMessage("i wonder why it's called langchain"),
    AIMessage(
        'Well, I guess they thought "WordRope" and "SentenceString" just didn\'t have the same ring to it!'
    ),
    HumanMessage("and who is harrison chasing anyways"),
    AIMessage(
        "Hmmm let me think.\n\nWhy, he's probably chasing after the last cup of coffee in the office!"
    ),
    HumanMessage("why is 42 always the answer?"),
    AIMessage(
        "Because it’s the only number that’s constantly right, even when it doesn’t add up!"
    ),
    HumanMessage("What did the cow say?"),
]


def message_processor(messages: list[BaseMessage]) -> list[BaseMessage]:
    """A sample message processor that:

    1. Keeps the system message
    2. Keeps up to max number of messages
    3. Make sure that the last message is a HumanMessage

    You will likely want to instead count based on tokens,
    and/or increase the number of messages.

    Please see the API reference for trim_messages for more details.

    https://python.langchain.com/api_reference/core/messages/langchain_core.messages.utils.trim_messages.html
    """
    return trim_messages(
        messages,
        token_counter=len,  # <-- Will just count the number of messages rather than tokens
        max_tokens=5,  # <-- allow up to 5 messages.
        strategy="last",
        include_system=True,
        allow_partial=False,
    )


message_processor(full_message_history)

[SystemMessage(content="you're a good assistant, you always respond with a joke.", additional_kwargs={}, response_metadata={}),
 AIMessage(content="Hmmm let me think.\n\nWhy, he's probably chasing after the last cup of coffee in the office!", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='why is 42 always the answer?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Because it’s the only number that’s constantly right, even when it doesn’t add up!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What did the cow say?', additional_kwargs={}, response_metadata={})]

</details>

## LCEL: Add a pre-processor in front of the chat model

The simplest way to add complex conversation management is by introducing a pre-processing step in front of the chat model and pass the full conversation history to the pre-processing step.

This approach is conceptually simple and will work in many situations; for example, if using a [RunnableWithMessageHistory](/docs/how_to/message_history/) instead of wrapping the chat model, wrap the chat model with the pre-processor.

The obvious downside of this approach is that latency starts to increase as the conversation history grows because of two reasons:

1. As the conversation gets longer, more data may need to be fetched from whatever store your'e using to store the conversation history (if not storing it in memory).
2. The pre-processing logic will end up doing a lot of redundant computation, repeating computation from previous steps of the conversation.

:::caution

If you're using tools, remember to bind the tools to the model before adding a pre-processing step to it!

:::

<details open>

In [15]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool

model = ChatOpenAI()


@tool
def what_did_the_cow_say() -> str:
    """Check to see what the cow said."""
    return "foo"

model_with_tools = model.bind_tools([what_did_the_cow_say])

# highlight-next-line
model_with_preprocessor = message_processor | model_with_tools

# full_message_history in the previous code block.
# We pass it explicity to the model_with_preprocesor for illustrative purposes.
# If you're using `RunnableWithMessageHistory` the history will be automatically
# read from the source the you configure.
model_with_preprocessor.invoke(full_message_history)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_g7l7AFBHnfInA9ps2DpnkrsU', 'function': {'arguments': '{}', 'name': 'what_did_the_cow_say'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 126, 'total_tokens': 142, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-3df002ca-da48-4b91-ad7f-c8e3ad5c7550-0', tool_calls=[{'name': 'what_did_the_cow_say', 'args': {}, 'id': 'call_g7l7AFBHnfInA9ps2DpnkrsU', 'type': 'tool_call'}], usage_metadata={'input_tokens': 126, 'output_tokens': 16, 'total_tokens': 142})

</details>

If you need to implement more efficient logic and want to use `RunnableWithMessageHistory` for now the way to achieve this
is to subclass from [BaseChatMessageHistory](https://api.python.langchain.com/en/latest/chat_history/langchain_core.chat_history.BaseChatMessageHistory.html) and
define appropriate logic for `add_messages` (that doesn't simply append the history, but instead re-writes it).

Unless you have a good reason to implement this solution, you should instead use LangGraph,

## LangGraph

### Agent Executor with a pre-built agent

If you're migrating off a pre-built langchain agent that uses memory.

You can create a pre-built agent using: [create_react_agent](https://langchain-ai.github.io/langgraph/reference/prebuilt/#create_react_agent).

To add memory pre-processing to the agent, you can do the following:

```python

...

# highlight-start
def state_modifier(state) -> list[BaseMessage]:
    """Given the agent state, return a list of messages for the chat model."""
    # We're using the message processor defined above.
    return message_processor(state['messages'])
# highlight-end    

app = create_react_agent(
    model,
    tools=[get_user_age], 
    checkpointer=memory,
    # highlight-next-line
    state_modifier=state_modifier
)

...

```

At each turn of the conversation, 

In [14]:
import uuid

from langchain_core.messages import HumanMessage
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent


@tool
def get_user_age(name: str) -> str:
    """Use this tool to find the user's age."""
    # This is a placeholder for the actual implementation
    if "bob" in name.lower():
        return "42 years old"
    return "41 years old"


memory = MemorySaver()
model = ChatOpenAI()


# highlight-start
def state_modifier(state) -> list[BaseMessage]:
    """Given the agent state, return a list of messages for the chat model."""
    # We're using the message processor defined above.
    return message_processor(state["messages"])


# highlight-end

app = create_react_agent(
    model,
    tools=[get_user_age],
    checkpointer=memory,
    # highlight-next-line
    state_modifier=state_modifier,
)

# The thread id is a unique key that identifies
# this particular conversation.
# We'll just generate a random uuid here.
thread_id = uuid.uuid4()
config = {"configurable": {"thread_id": thread_id}}

# Tell the AI that our name is Bob, and ask it to use a tool to confirm
# that it's capable of working like an agent.
input_message = HumanMessage(content="hi! I'm bob. What is my age?")

for event in app.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

# Confirm that the chat bot has access to previous conversation
# and can respond to the user saying that the user's name is Bob.
input_message = HumanMessage(content="do you remember my name?")

for event in app.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

hi! I'm bob. What is my age?
================================== Ai Message ==================================
Tool Calls:
  get_user_age (call_kTEpBUbRFbKE3DZolG9tFrgD)
 Call ID: call_kTEpBUbRFbKE3DZolG9tFrgD
  Args:
    name: bob
================================= Tool Message =================================
Name: get_user_age

42 years old
================================== Ai Message ==================================

Bob, you are 42 years old.
================================ Human Message =================================

do you remember my name?
================================== Ai Message ==================================

Yes, your name is Bob.


### ConversationSummaryMemory / ConversationSummaryBufferMemory

It’s essential to summarize conversations efficiently to prevent growing latency as the conversation history grows.

Please follow the guide [how to add summary of the conversation history](https://langchain-ai.github.io/langgraph/how-tos/memory/add-summary-conversation-history/) to see learn how to
handle conversation summarization efficiently with LangGraph.

## Next steps

Explore persistence with LangGraph:

* [LangGraph quickstart tutorial](https://langchain-ai.github.io/langgraph/tutorials/introduction/)
* [How to add persistence ("memory") to your graph](https://langchain-ai.github.io/langgraph/how-tos/persistence/)
* [How to manage conversation history](https://langchain-ai.github.io/langgraph/how-tos/memory/manage-conversation-history/)
* [How to add summary of the conversation history](https://langchain-ai.github.io/langgraph/how-tos/memory/add-summary-conversation-history/)

Add persistence with simple LCEL (favor langgraph for more complex use cases):

* [How to add message history](/docs/how_to/message_history/)

Working with message history:

* [How to trim messages](/docs/how_to/trim_messages)
* [How to filter messages](/docs/how_to/filter_messages/)
* [How to merge message runs](/docs/how_to/merge_message_runs/)